- Contains setting up GPIO button as input and having it generate an interrupt
- Also contains an asyncio architecture for everything 

In [ ]:
from pynq import Overlay, interrupt
from pynq.lib import axigpio
from IPython.display import clear_output

import asyncio
import time

# LOad overlay
overlay = Overlay('MainDesign.bit')

In [ ]:
overlay?

In [ ]:
gpio_ip = overlay.axi_gpio_0
gpio_ip.setdirection("in")
gpio_interrupt = gpio_ip.ip2intc_irpt

# Access the register map
gpio_register_map = gpio_ip.register_map

print(gpio_ip._interrupts)
print(gpio_ip.register_map)
overlay.interrupt_pins

In [ ]:
interrupt1 = interrupt.Interrupt('axi_gpio_0/ip2intc_irpt')
async def GPIO_handler():
    await interrupt1.wait()
    while True:
        if gpio_ip.read() == 1:
            print("GPIO Interrupt occurred!")
            
            # Clearing the interrupt
            gpio_register_map.IP_ISR.Channel_1_Interrupt_Status = 1
            break
            

In [ ]:
if __name__ == '__main__':
    # Main asyncio loop
    loop = asyncio.get_event_loop()
    
    # Enable the GPIO interrupt
    gpio_register_map.IP_IER.Channel_1_Interrupt_Enable = 1
    try:
        # Run the interrupt handler coroutine until
        loop.run_until_complete(GPIO_handler())
    except KeyboardInterrupt:
        print("Script interrupted by user.")